<a href="https://colab.research.google.com/github/iamVaibhav100/text_analysis_from_extracted_articles/blob/main/blackcoffer_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install textstat

In [ ]:
# !pip install textblob

In [ ]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from textstat.textstat import textstat
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
nltk.download('omw-1.4')

In [ ]:
# Read in the Excel sheet
df = pd.read_excel("drive/MyDrive/blackcoffer_assignment/Input.xlsx")

In [ ]:
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
# # Iterate through each row of the sheet
# for index, row in df.iterrows():
#     url_id = row['URL_ID']
#     url = row['URL']

#     # Make a request to the website
#     page = requests.get(url, headers=headers)

#     # Create a BeautifulSoup object to parse the HTML
#     soup = BeautifulSoup(page.content, 'html.parser')

#     # Extract the title and text of the article
#     title_h1 = soup.find('h1')
#     text_div = soup.find('div', class_='td-post-content')
#     if text_div and title_h1:
#         text = text_div.text
#         title = title_h1.text
#     else:
#         text = ''
#         title = 'Not found!'

#     # Write the title and text to a new file with the file name being the URL_ID
#     with open(f"drive/MyDrive/blackcoffer_assignment/extracted/{url_id}.txt", "w") as file:
#         file.write(title + '\n\n')
#         file.write(text)

In [ ]:
# Load the stopwords from the folder
import codecs
stopwords = set()
stopwords_folder = "drive/MyDrive/blackcoffer_assignment/stop_words"
for file in os.listdir(stopwords_folder):
    if file.endswith(".txt"):
        with codecs.open(os.path.join(stopwords_folder, file), "r", encoding='utf-8', errors='ignore') as f:
            stopwords.update(f.read().lower().split())

In [ ]:
list(stopwords)[:5], len(stopwords)

(['padgett', 'mckee', 'warden', 'bradford', 'kohler'], 12754)

In [ ]:
# Load the positive and negative words
positive_words = set()
with codecs.open("drive/MyDrive/blackcoffer_assignment/dictionary/positive-words.txt", "r", encoding='utf-8', errors='ignore') as f:
    positive_words.update(f.read().split())
negative_words = set()
with codecs.open("drive/MyDrive/blackcoffer_assignment/dictionary/negative-words.txt", "r", encoding='utf-8', errors='ignore') as f:
    negative_words.update(f.read().split())

In [ ]:
list(positive_words)[:5], list(negative_words)[:5]

(['beneficial', 'enthral', 'courtly', 'eye-catching', 'humorous'],
 ['forfeit', 'repudiation', 'wripped', 'misguidance', 'strangely'])

In [ ]:
# Initialize the dataframe to store the results
output = pd.DataFrame(columns=['URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])

In [ ]:
from textblob import TextBlob

In [ ]:
# article_names = ["100.0.txt", "101.0.txt", "103.0.txt", "104.0.txt", "105.0.txt"]
# for file in article_names:
#     if file.endswith(".txt"):
#         with open(os.path.join("drive/MyDrive/blackcoffer_assignment/extracted", file), "r") as f:
#             text = f.read()
#     # Clean the text by removing stopwords
#     text = " ".join([word for word in word_tokenize(text) if word.lower() not in stopwords])
#     text = text.lower()

#     positive_score = 0
#     negative_score = 0
#     polarity_score = 0
#     # Perform sentiment analysis using nltk library
#     sid = SentimentIntensityAnalyzer()
#     ss = sid.polarity_scores(text)
#     positive_score = ss['pos']
#     negative_score = ss['neg']
#     polarity_score = ss['compound']

#     # Perform TextBlob analysis
#     subjectivity_score = 0
#     tb = TextBlob(text)
#     subjectivity_score = tb.sentiment.subjectivity

#     # Tokenize the text
#     words = word_tokenize(text)

#     # Remove the stop words from the text
#     filtered_words = [word for word in words if word.lower() not in stopwords]
#     text = ' '.join(filtered_words)

#     # Lemmatize the words
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

#     # Calculate the average sentence length
#     avg_sentence_length = 0
#     sentences = sent_tokenize(text)
#     avg_sentence_length = sum(len(sentence.split()) for sentence in sentences) / len(sentences)

#     complex_word_count = 0
#     percentage_of_complex_words = 0
#     for word in word_tokenize(text):
#         if textstat.syllable_count(word) >= 3:
#                 complex_word_count += 1
#     percentage_of_complex_words = (complex_word_count / len(word_tokenize(text))) * 100

#     word_count = len(word_tokenize(text))
#     syllable_per_word = textstat.syllable_count(text) / word_count

#     fog_index = 0
#     fog_index = 0.4 * ((avg_sentence_length + percentage_of_complex_words))

#     avg_number_of_words_per_sentence = 0
#     avg_words_per_sentence = len(word_tokenize(text)) / len(sentences)

#     personal_pronouns = ["I", "me", "mine", "my", "we", "us", "our", "ours"]
#     personal_pronoun_count = 0
#     for word in word_tokenize(text):
#         if word.lower() in personal_pronouns:
#             personal_pronoun_count += 1

#     avg_word_length = sum(len(word) for word in word_tokenize(text)) / len(word_tokenize(text))


In [ ]:
for file in os.listdir("drive/MyDrive/blackcoffer_assignment/extracted"):
    if file.endswith(".txt"):
        with open(os.path.join("drive/MyDrive/blackcoffer_assignment/extracted", file), "r") as f:
            text = f.read()
            # Clean the text by removing stopwords
            text = " ".join([word for word in word_tokenize(text) if word.lower() not in stopwords])
            text = text.lower()

            # Clean the text by removing stopwords
            text = " ".join([word for word in word_tokenize(text) if word.lower() not in stopwords])
            text = text.lower()

            positive_score = 0
            negative_score = 0
            polarity_score = 0
            # Perform sentiment analysis using nltk library
            sid = SentimentIntensityAnalyzer()
            ss = sid.polarity_scores(text)
            positive_score = ss['pos']
            negative_score = ss['neg']
            polarity_score = ss['compound']

            # Perform TextBlob analysis
            subjectivity_score = 0
            tb = TextBlob(text)
            subjectivity_score = tb.sentiment.subjectivity

            # Tokenize the text
            words = word_tokenize(text)

            # Remove the stop words from the text
            filtered_words = [word for word in words if word.lower() not in stopwords]
            text = ' '.join(filtered_words)

            # Lemmatize the words
            lemmatizer = WordNetLemmatizer()
            lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

            # Calculate the average sentence length
            avg_sentence_length = 0
            sentences = sent_tokenize(text)
            avg_sentence_length = sum(len(sentence.split()) for sentence in sentences) / len(sentences)

            complex_word_count = 0
            percentage_of_complex_words = 0
            for word in word_tokenize(text):
                if textstat.syllable_count(word) >= 3:
                        complex_word_count += 1
            percentage_of_complex_words = (complex_word_count / len(word_tokenize(text))) * 100

            word_count = len(word_tokenize(text))
            syllable_per_word = textstat.syllable_count(text) / word_count

            fog_index = 0
            fog_index = 0.4 * ((avg_sentence_length + percentage_of_complex_words))

            avg_number_of_words_per_sentence = 0
            avg_words_per_sentence = len(word_tokenize(text)) / len(sentences)

            personal_pronouns = ["I", "me", "mine", "my", "we", "us", "our", "ours"]
            personal_pronoun_count = 0
            for word in word_tokenize(text):
                if word.lower() in personal_pronouns:
                    personal_pronoun_count += 1

            avg_word_length = sum(len(word) for word in word_tokenize(text)) / len(word_tokenize(text))

    output.append([file, positive_score, negative_score, polarity_score, subjectivity_score,
            avg_sentence_length, percentage_of_complex_words, fog_index, avg_words_per_sentence,
            complex_word_count, word_count, syllable_per_word, personal_pronoun_count, avg_word_length])

In [ ]:
output.to_csv("drive/MyDrive/blackcoffer_assignment/results/text_analysis.csv", index=False)